In [35]:
import pystac_client
import odc.stac

In [36]:
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

In [37]:
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

In [40]:
# Set a bounding box
# [xmin, ymin, xmax, ymax] in latitude and longitude
bbox = [153.3, -27.5, 153.4, -27.6]

# Set a start and end date
start_date = "1985-12-31"
end_date = "1987-01-01"

# Set product ID as the STAC "collection"
collections = ["ga_ls_fc_pc_cyear_3"]

In [41]:
# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")

items

Found: 2 datasets


[<Item id=41d9b9e6-c168-436a-b639-f2446f508d6e>,
 <Item id=b3c3be4c-eb3b-47f4-a130-4bd00f031bc4>]

In [29]:
ds = odc.stac.load(
             items=items,
             crs="EPSG:3577",
             lat=(bbox[1], bbox[3]),
             lon=(bbox[0], bbox[2]),
             time=("2015-01-01", "2018-12-31"))

ds


ValueError: Failed to auto-guess CRS/resolution.

In [ ]:
# Print the loaded dataset
print("Dataset info:")
print(ds)

# Print the type of the bs_pc_10 data
print("\nType of bs_pc_10:")
print(type(ds.bs_pc_10))

# Print the data type (dtype) of the band values
print("\nData type of bs_pc_10 values:")
print(ds.bs_pc_10.dtype)

Dataset info:
<xarray.Dataset> Size: 26MB
Dimensions:      (y: 424, x: 384, time: 4)
Coordinates:
  * y            (y) float64 3kB -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 3kB 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 4B 3577
  * time         (time) datetime64[ns] 32B 2015-01-01 2016-01-01 ... 2018-01-01
Data variables:
    qa           (time, y, x) float32 3MB 0.0 0.0 0.0 0.0 ... 2.0 2.0 2.0 2.0
    bs_pc_10     (time, y, x) float32 3MB nan nan nan nan ... 3.0 4.0 4.0 3.0
    bs_pc_50     (time, y, x) float32 3MB nan nan nan nan ... 4.0 5.0 6.0 6.0
    bs_pc_90     (time, y, x) float32 3MB nan nan nan nan ... 29.0 25.0 14.0 9.0
    pv_pc_10     (time, y, x) float32 3MB nan nan nan nan ... 39.0 54.0 60.0
    pv_pc_50     (time, y, x) float32 3MB nan nan nan nan ... 75.0 66.0 66.0
    pv_pc_90     (time, y, x) float32 3MB nan nan nan nan ... 78.0 75.0 77.0
    npv_pc_10    (time, y, x) float32 3MB nan nan nan nan ... 15.0 19.0

In [ ]:
ds.bs_pc_10[0]

<xarray.DataArray 'bs_pc_10' (y: 424, x: 384)> Size: 651kB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [25., 25., 35., ...,  4.,  3.,  3.],
       [28., 22., 36., ...,  5.,  4.,  2.],
       [20., 28., 34., ...,  5.,  3.,  3.]], dtype=float32)
Coordinates:
  * y            (y) float64 3kB -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 3kB 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 4B 3577
    time         datetime64[ns] 8B 2015-01-01

In [ ]:
ds.bs_pc_10[1]

<xarray.DataArray 'bs_pc_10' (y: 424, x: 384)> Size: 651kB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [24., 23., 22., ...,  3.,  3.,  3.],
       [28., 18., 26., ...,  3.,  3.,  2.],
       [21., 22., 31., ...,  4.,  3.,  2.]], dtype=float32)
Coordinates:
  * y            (y) float64 3kB -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 3kB 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 4B 3577
    time         datetime64[ns] 8B 2016-01-01

In [ ]:
ds.bs_pc_10[0][0]

<xarray.DataArray 'bs_pc_10' (x: 384)> Size: 2kB
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan,  2.,  2.,  2.,  2.,  5.,  9., nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  3.,  2.,  2.,  3.,  3.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  3.,  3.,  2.,  2.,  2.,  2., nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan], dtype=float32)
Coordinates:
    y            float64 8B -3.156e+06
  * x            (x) float64 3kB 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 4B 3577
    time         datetime64[ns] 8B 2015-01-01

In [ ]:
bbox = [146.29, -39.10, 146.39, -39.00]

# Set a start and end date
start_date = "2015-01-01"
end_date = "2020-12-31"

# Set product ID as the STAC "collection"
collections = ["ga_ls_fc_pc_cyear_3"]

# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")


ds = odc.stac.load(
             items=items,
             crs="EPSG:3577",
             lat=(bbox[1], bbox[3]),
             lon=(bbox[0], bbox[2]),
             time=("2015-01-01", "2018-12-31"))

ds

Found: 12 datasets


<xarray.Dataset> Size: 32MB
Dimensions:      (y: 397, x: 331, time: 6)
Coordinates:
  * y            (y) float64 3kB -4.334e+06 -4.334e+06 ... -4.346e+06 -4.346e+06
  * x            (x) float64 3kB 1.248e+06 1.248e+06 ... 1.257e+06 1.257e+06
    spatial_ref  int32 4B 3577
  * time         (time) datetime64[ns] 48B 2015-01-01 2016-01-01 ... 2020-01-01
Data variables:
    qa           (time, y, x) float32 3MB 0.0 0.0 0.0 0.0 ... 2.0 2.0 2.0 2.0
    bs_pc_10     (time, y, x) float32 3MB nan nan nan nan ... 2.0 2.0 4.0 4.0
    bs_pc_50     (time, y, x) float32 3MB nan nan nan nan ... 4.0 3.0 5.0 7.0
    bs_pc_90     (time, y, x) float32 3MB nan nan nan nan ... 7.0 7.0 8.0 10.0
    pv_pc_10     (time, y, x) float32 3MB nan nan nan nan ... 76.0 74.0 73.0
    pv_pc_50     (time, y, x) float32 3MB nan nan nan nan ... 83.0 80.0 79.0
    pv_pc_90     (time, y, x) float32 3MB nan nan nan nan ... 85.0 82.0 83.0
    npv_pc_10    (time, y, x) float32 3MB nan nan nan nan ... 7.0 7.0 9.0 8.0
    npv_pc_50    (time, y, x) float32 3MB nan nan nan nan ... 12.0 13.0 11.0
    npv_pc_90    (time, y, x) float32 3MB nan nan nan nan ... 21.0 20.0 17.0

In [ ]:
ds.bs_pc_10.__len__()

6

In [ ]:
ds.bs_pc_10[5]

<xarray.DataArray 'bs_pc_10' (y: 397, x: 331)> Size: 526kB
array([[nan, nan, nan, ...,  2.,  2.,  2.],
       [nan, nan, nan, ...,  2.,  2.,  2.],
       [nan, nan, nan, ...,  2.,  2.,  2.],
       ...,
       [nan, nan, nan, ...,  2.,  2.,  2.],
       [nan, nan, nan, ...,  2.,  2.,  2.],
       [nan, nan, nan, ...,  2.,  4.,  4.]], dtype=float32)
Coordinates:
  * y            (y) float64 3kB -4.334e+06 -4.334e+06 ... -4.346e+06 -4.346e+06
  * x            (x) float64 3kB 1.248e+06 1.248e+06 ... 1.257e+06 1.257e+06
    spatial_ref  int32 4B 3577
    time         datetime64[ns] 8B 2020-01-01

In [ ]:
ds.bs_pc_10[5][5]

<xarray.DataArray 'bs_pc_10' (x: 331)> Size: 1kB
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  7.,  9.,  8.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,
        7.,  8., 13., 10., 10.,  9.,  5.,  5.,  3.,  2.,  2.,  2.,  2.,
        3.,  2.,  4.,  3.,  3.,  4.,  6.,  9., 11., 10.,  9.,  7.,  4.,
        6., 10., 13., 15., 14., 10.,  6.,  3.,  2.,  2.,  5.,  3.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  2.,  3.,
        3.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  3.,  3.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        3.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.], dtype=float32)
Coordinates:
    y            float64 8B -4.335e+06
  * x            (x) float64 3kB 1.248e+06 1.248e+06 ... 1.257e+06 1.257e+06
    spatial_ref  int32 4B 3577
    time         datetime64[ns] 8B 2020-01-01

In [ ]:
ds.pv_pc_50[5][5]

<xarray.DataArray 'pv_pc_50' (x: 331)> Size: 1kB
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 30., 18., 32.,
       44., 47., 53., 57., 62., 61., 65., 65., 59., 64., 65., 64., 61.,
       60., 63., 66., 64., 64., 66., 66., 64., 64., 68., 70., 69., 71.,
       72., 68., 66., 69., 72., 76., 78., 80., 81., 81., 83., 81., 78.,
       79., 80., 78., 76., 77., 75., 70., 70., 69., 77., 82., 80., 70.,
       56., 53., 57., 55., 53., 57., 59., 56., 60., 68., 69., 66., 70.,
       72., 70., 64., 59., 55., 53., 52., 47., 43., 44., 49., 54., 55.,
       51., 47., 42., 40., 46., 48., 52., 59., 60., 60., 59., 62., 68.,
       76., 81., 72., 83., 74., 83., 83., 83., 83., 83., 77., 73., 73.,
       78., 76., 78., 83., 82., 79., 74., 71., 67., 66., 72., 75., 81.,
       75., 67., 66., 67., 67., 73., 79., 81., 87., 93., 92., 87., 82.,
       82., 86., 89., 86., 84., 85., 85., 84., 91., 96., 96., 97., 97.,
       97., 96., 96., 96., 96., 97., 97., 97., 95., 95., 96., 96., 96.,
       94., 91., 94., 91., 91., 94., 96., 95., 95., 95., 97., 96., 95.,
       95., 94., 94., 94., 96., 95., 96., 95., 94., 95., 97., 96., 93.,
       92., 95., 96., 96., 96., 95., 97., 97., 96., 95., 91., 91., 95.,
       95., 94., 93., 93., 93., 94., 93., 95., 95., 94., 93., 93., 95.,
       95., 92., 93., 96., 96., 94., 90., 95., 97., 96., 93., 91., 92.,
       93., 94., 95., 94., 93., 93., 92., 90., 89., 88., 87., 88., 91.,
       94., 96., 95., 94., 93., 93., 92., 93., 91., 90., 88., 88., 90.,
       93., 93., 93., 96., 93., 91., 90., 87., 83., 84., 87., 86., 83.,
       82., 78., 75., 78., 80., 79., 76., 77., 75., 77., 77., 80., 82.,
       81., 80., 81., 82., 84., 85., 90., 93., 90., 86., 84., 81., 79.,
       79., 79., 76., 76., 75., 73., 76., 76., 79., 84., 86., 88., 87.,
       88., 88., 86., 87., 91., 93., 93., 92., 91., 90., 91., 87., 88.,
       93., 93., 90., 89., 93., 95.], dtype=float32)
Coordinates:
    y            float64 8B -4.335e+06
  * x            (x) float64 3kB 1.248e+06 1.248e+06 ... 1.257e+06 1.257e+06
    spatial_ref  int32 4B 3577
    time         datetime64[ns] 8B 2020-01-01

In [ ]:
ds.pv_pc_50[5].mean().values

array(76.368935, dtype=float32)

In [ ]:
for item in ds:

    if item == 'qa':
        continue

    mean = ds[item].mean().values

    print(f"Mean value of {item}: {mean}")

Mean value of bs_pc_10: 3.305572986602783
Mean value of bs_pc_50: 5.285403251647949
Mean value of bs_pc_90: 8.476634979248047
Mean value of pv_pc_10: 69.29605102539062
Mean value of pv_pc_50: 75.7697982788086
Mean value of pv_pc_90: 81.52720642089844
Mean value of npv_pc_10: 10.704602241516113
Mean value of npv_pc_50: 16.414344787597656
Mean value of npv_pc_90: 21.93483543395996
